In [0]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

#creating the dataframe df1
data1 = [(1,'Jhon',17),(2,'Maria',20),(3,'Raj',None),(4,'Rachel',18)]
columns = ["id","name","age"]
df1 = spark.createDataFrame(data1,columns)
df1.show()

+---+------+----+
| id|  name| age|
+---+------+----+
|  1|  Jhon|  17|
|  2| Maria|  20|
|  3|   Raj|NULL|
|  4|Rachel|  18|
+---+------+----+



In [0]:
# Count null entries in each column
null_counts = df1.select([sum(col(c).isNull().cast("int")).alias(c) for c in df1.columns])

null_counts.show()

+---+----+---+
| id|name|age|
+---+----+---+
|  0|   0|  1|
+---+----+---+



In [0]:
#Remove the row with null entires and store them in a new dataframe named df2
df2 = df1.filter(col("age").isNull())
df2.show()

+---+----+----+
| id|name| age|
+---+----+----+
|  3| Raj|NULL|
+---+----+----+



In [0]:
#create a new dataframe df3
data2 = [(1,'seatle',82),(2,'london',75),(3,'banglore',60),(4,'boston',90)]
columns2 = ["id","city","code"]

df3 = spark.createDataFrame(data2,columns2)
df3.show()

mergedf = df1.join(df3, df1["id"]==df3["id"],"inner").select(df1["id"],"name","age","city","code")
mergedf.show()

+---+--------+----+
| id|    city|code|
+---+--------+----+
|  1|  seatle|  82|
|  2|  london|  75|
|  3|banglore|  60|
|  4|  boston|  90|
+---+--------+----+

+---+------+----+--------+----+
| id|  name| age|    city|code|
+---+------+----+--------+----+
|  1|  Jhon|  17|  seatle|  82|
|  2| Maria|  20|  london|  75|
|  3|   Raj|NULL|banglore|  60|
|  4|Rachel|  18|  boston|  90|
+---+------+----+--------+----+



In [0]:
#fill the null value with the mean age of students
#calculate the mean age
meanage = mergedf.select(round(mean("age"))).collect()[0][0]
print(meanage)

filldf = mergedf.na.fill({"age":meanage})
filldf.show()

18.0
+---+------+---+--------+----+
| id|  name|age|    city|code|
+---+------+---+--------+----+
|  1|  Jhon| 17|  seatle|  82|
|  2| Maria| 20|  london|  75|
|  3|   Raj| 18|banglore|  60|
|  4|Rachel| 18|  boston|  90|
+---+------+---+--------+----+



In [0]:
#Get the students who are 18 years or older
filterdf = filldf.filter(col("age")>= 18)
filterdf.show()

+---+------+---+--------+----+
| id|  name|age|    city|code|
+---+------+---+--------+----+
|  2| Maria| 20|  london|  75|
|  3|   Raj| 18|banglore|  60|
|  4|Rachel| 18|  boston|  90|
+---+------+---+--------+----+

